In [75]:
#imports 

import numpy as np
import pandas as pd 
import seaborn as sns 
import openpyxl
import pickle 

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import classification_report


In [30]:
df = pd.read_excel('C:\\Users\\Malavi\\Desktop\\RFISUBCODDES Predictor\\Ancillary Rev - 2015 - 2023.xlsx', header=2)
df


,Flight Date,Point of Sale,Channel,RFISUBCOD,RFISUBCODDES,RFICOD,REMARKS,Count,Sum of Revenue USD
0,2015-01-01,Sri Lanka,ATO/CTO/GSA/PSA,0BJ,UPGRADE,Air transportation,UPGRADE,2,1223.78
1,2015-01-01,Sri Lanka,Call centre,0BJ,UPGRADE,Air transportation,UPGRADE,61,13516.32
2,2015-01-01,Hong Kong,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,4,384.23
3,2015-01-01,India Tamilnadu - Chennai,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,26,1658.70
4,2015-01-01,India Trivandrum,ATO/CTO/GSA/PSA,0IK,EXCESS WEIGHT,Baggage,EXCESS WEIGHT,6,272.64
...,...,...,...,...,...,...,...,...,...
149997,2020-01-07,India Northern,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,2,4.06
149998,2020-01-07,Singapore,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,12.60
149999,2020-01-07,Sri Lanka,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,1,3.04
150000,2020-01-07,United Kingdom,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,25.08


In [31]:
df.isnull().sum()


Flight Date             0
Point of Sale           0
Channel                 0
RFISUBCOD               0
RFISUBCODDES            0
RFICOD                  0
REMARKS               688
Count                   0
Sum of Revenue USD      1
dtype: int64

In [32]:
df = df.dropna()
df.shape

(149313, 9)

In [33]:
df.isnull().sum()

Flight Date           0
Point of Sale         0
Channel               0
RFISUBCOD             0
RFISUBCODDES          0
RFICOD                0
REMARKS               0
Count                 0
Sum of Revenue USD    0
dtype: int64

In [34]:
# Convert Flight Date to datetime and extract Year & Month


df['Flight Date'] = pd.to_datetime(df['Flight Date'])
df['Year'] = df['Flight Date'].dt.year
df['Month'] = df['Flight Date'].dt.month



C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2572032901.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Flight Date'] = pd.to_datetime(df['Flight Date'])
C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2572032901.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year'] = df['Flight Date'].dt.year
C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\2572032901.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [35]:
df.tail(5)

,Flight Date,Point of Sale,Channel,RFISUBCOD,RFISUBCODDES,RFICOD,REMARKS,Count,Sum of Revenue USD,Year,Month
149997,2020-01-07,India Northern,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,2,4.06,2020,1
149998,2020-01-07,Singapore,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,12.60,2020,1
149999,2020-01-07,Sri Lanka,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,1,3.04,2020,1
150000,2020-01-07,United Kingdom,IBE,0EO,CARBON OFFSET,Financial Impact,CARBON OFFSET,5,25.08,2020,1
150001,2020-01-07,India Northern,ATO/CTO/GSA/PSA,0IJ,EXCESS PIECE,Baggage,EXCESS WEIGHT,3,98.95,2020,1


In [36]:
df = df.drop(['REMARKS', 'RFICOD','Channel','Flight Date'], axis=1)

In [37]:
df.head()

,Point of Sale,RFISUBCOD,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month
0,Sri Lanka,0BJ,UPGRADE,2,1223.78,2015,1
1,Sri Lanka,0BJ,UPGRADE,61,13516.32,2015,1
2,Hong Kong,0IK,EXCESS WEIGHT,4,384.23,2015,1
3,India Tamilnadu - Chennai,0IK,EXCESS WEIGHT,26,1658.70,2015,1
4,India Trivandrum,0IK,EXCESS WEIGHT,6,272.64,2015,1


In [38]:
df = df.drop('RFISUBCOD', axis=1)

In [39]:
df.head()

,Point of Sale,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month
0,Sri Lanka,UPGRADE,2,1223.78,2015,1
1,Sri Lanka,UPGRADE,61,13516.32,2015,1
2,Hong Kong,EXCESS WEIGHT,4,384.23,2015,1
3,India Tamilnadu - Chennai,EXCESS WEIGHT,26,1658.70,2015,1
4,India Trivandrum,EXCESS WEIGHT,6,272.64,2015,1


In [40]:
df.tail()

,Point of Sale,RFISUBCODDES,Count,Sum of Revenue USD,Year,Month
149997,India Northern,CARBON OFFSET,2,4.06,2020,1
149998,Singapore,CARBON OFFSET,5,12.60,2020,1
149999,Sri Lanka,CARBON OFFSET,1,3.04,2020,1
150000,United Kingdom,CARBON OFFSET,5,25.08,2020,1
150001,India Northern,EXCESS PIECE,3,98.95,2020,1


In [41]:
df.isnull().sum()

Point of Sale         0
RFISUBCODDES          0
Count                 0
Sum of Revenue USD    0
Year                  0
Month                 0
dtype: int64

In [42]:
df.to_csv('cleaned_data.csv', index=False)


In [43]:
df.describe()

,Count,Sum of Revenue USD,Year,Month
count,149313.000000,149313.000000,149313.000000,149313.000000
mean,3.920449,347.589336,2017.671924,6.747202
std,6.963465,1212.119752,1.302187,3.551077
min,1.000000,0.000000,2015.000000,1.000000
25%,1.000000,29.780000,2017.000000,4.000000
50%,2.000000,83.030000,2018.000000,7.000000
75%,4.000000,244.900000,2019.000000,10.000000
max,193.000000,46906.530000,2020.000000,12.000000


In [47]:
# Step 1: Compute revenue per transaction
df['Revenue_per_Count'] = df['Sum of Revenue USD'] / df['Count']

# Step 2: Find index of max Revenue_per_Count per group
idx = df.groupby(['Point of Sale', 'Month', 'Year'])['Revenue_per_Count'].idxmax()

# Step 3: Create a new column and assign best ancillary only at those idx positions
df['Best_Ancillary'] = np.nan
df.loc[idx, 'Best_Ancillary'] = df.loc[idx, 'RFISUBCODDES']

# Step 4: Forward-fill and back-fill within each group to populate the rest
df['Best_Ancillary'] = df.groupby(['Point of Sale', 'Month', 'Year'])['Best_Ancillary'].ffill().bfill()

C:\Users\Malavi\AppData\Local\Temp\ipykernel_21888\937778612.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['EXCESS WEIGHT' 'EXCESS PIECE' 'UPGRADE' ...
 'PRE-RESERVED SEAT ASSIGNMENT' 'PRE PAID BAGGAGE' 'PRE PAID BAGGAGE']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, 'Best_Ancillary'] = df.loc[idx, 'RFISUBCODDES']


In [52]:
df.isnull().sum()

Point of Sale         0
RFISUBCODDES          0
Count                 0
Sum of Revenue USD    0
Year                  0
Month                 0
Revenue_per_Count     0
Best_Ancillary        0
dtype: int64

In [53]:
df['Month_sin'] = np.sin(2 * np.pi * (df['Month'] - 1) / 12)
df['Month_cos'] = np.cos(2 * np.pi * (df['Month'] - 1) / 12)

In [54]:
le_pos = LabelEncoder()
le_anc = LabelEncoder()

df['Point_of_Sale_encoded'] = le_pos.fit_transform(df['Point of Sale'])
df['RFISUBCODDES_encoded'] = le_anc.fit_transform(df['RFISUBCODDES'])
df['Target_encoded'] = le_anc.transform(df['Best_Ancillary'])

In [90]:
features = [
    'Point_of_Sale_encoded',
    'Month_sin',
    'Month_cos',
    'Year',
    'Count',
    'Revenue_per_Count'
]
target = 'RFISUBCODDES_encoded'



In [91]:

X = df[features]
y = df[target]


In [118]:
from imblearn.over_sampling import SMOTE
from collections import Counter

# Check class distribution
class_counts = Counter(y)
print("Current class counts:", class_counts)

# Identify classes with samples < n_neighbors+1 (default=6)
small_classes = [cls for cls, count in class_counts.items() if count < 6]

# Custom SMOTE parameters per class
sampling_strategy = {
    cls: max(count, 6) for cls, count in class_counts.items()  # Ensure minimum 6 samples
}
smote = SMOTE(
    random_state=42,
    sampling_strategy=sampling_strategy,
    k_neighbors=min(5, min(class_counts.values()) - 1  # Auto-adjust n_neighbors
))

X_resampled, y_resampled = smote.fit_resample(X, y)

Current class counts: Counter({30: 31530, 7: 26803, 31: 26657, 26: 17724, 5: 14660, 41: 8790, 46: 4275, 3: 3875, 0: 3423, 9: 2750, 36: 2172, 45: 2102, 29: 1781, 39: 697, 47: 556, 24: 360, 4: 297, 2: 207, 50: 197, 33: 173, 28: 76, 38: 50, 35: 50, 19: 50, 40: 50, 18: 50, 22: 50, 44: 44, 25: 43, 12: 36, 11: 36, 21: 36, 32: 36, 14: 36, 8: 36, 23: 36, 1: 36, 17: 36, 34: 36, 43: 36, 16: 36, 20: 36, 48: 36, 15: 36, 37: 29, 42: 20, 13: 20, 10: 11, 51: 8, 27: 8, 6: 7, 49: 6})


In [119]:
print("Resampled class counts:", Counter(y_resampled))

Resampled class counts: Counter({30: 31530, 7: 26803, 31: 26657, 26: 17724, 5: 14660, 41: 8790, 46: 4275, 3: 3875, 0: 3423, 9: 2750, 36: 2172, 45: 2102, 29: 1781, 39: 697, 47: 556, 24: 360, 4: 297, 2: 207, 50: 197, 33: 173, 28: 76, 38: 50, 35: 50, 19: 50, 40: 50, 18: 50, 22: 50, 44: 44, 25: 43, 12: 36, 11: 36, 21: 36, 32: 36, 14: 36, 8: 36, 23: 36, 1: 36, 17: 36, 34: 36, 43: 36, 16: 36, 20: 36, 48: 36, 15: 36, 37: 29, 42: 20, 13: 20, 10: 11, 51: 8, 27: 8, 6: 7, 49: 6})


In [120]:
# Split into train/test sets
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, 
    test_size=0.2,
    random_state=42,
    stratify=y_resampled  # Now safe because SMOTE balances classes
)


In [121]:
# Train your model
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

C:\Users\Malavi\AppData\Roaming\Python\Python313\site-packages\xgboost\training.py:183: UserWarning: [09:06:08] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [123]:
def predict_top_ancillaries(year, month, point_of_sale):
    # Encode point of sale
    try:
        pos_encoded = le_pos.transform([point_of_sale])[0]
    except:
        return "Point of sale not found in our system"
    
    # Prepare input features
    month_sin = np.sin(2 * np.pi * (month - 1) / 12)
    month_cos = np.cos(2 * np.pi * (month - 1) / 12)
    
    # Use median values for Count & Revenue_per_Count (since they're unknown in predictions)
    input_data = pd.DataFrame({
        'Point_of_Sale_encoded': [pos_encoded],
        'Month_sin': [month_sin],
        'Month_cos': [month_cos],
        'Year': [year],
        'Count': [df['Count'].median()],
        'Revenue_per_Count': [df['Revenue_per_Count'].median()]
    })
    
    # Get probabilities for all ancillary services
    probs = model.predict_proba(input_data)[0]
    
    # Get top 3 predictions
    top3_indices = np.argsort(probs)[-3:][::-1]  # Indices sorted by highest probability
    top3_ancillaries = []
    for idx in top3_indices:
        ancillary = le_anc.inverse_transform([idx])[0]  # Convert back to original name
        probability = probs[idx]
        top3_ancillaries.append((ancillary, probability))
    
    return top3_ancillaries

In [128]:
year = 2025
month = 5
point_of_sale = "Australia"

top3 = predict_top_ancillaries(year, month, point_of_sale)
print(f"Top 3 predicted ancillary services for {point_of_sale} in {month}/{year}:")
for i, (ancillary, prob) in enumerate(top3, 1):
    print(f"{i}. {ancillary} (probability: {prob:.2%})")

Top 3 predicted ancillary services for Australia in 5/2025:
1. UNSPECIFIED TRANSPORATION (probability: 54.00%)
2. FLYSMILES UPGRADE AT AIPROT (probability: 31.38%)
3. SCUBA EQUIPMENT (probability: 7.87%)


In [122]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2%}")

Model Accuracy: 80.46%
